In [3]:
import os
import shutil
import pandas as pd
import numpy as np

### Data preprocessing

#### Combine all gene count files into one file

In [ ]:
# The downloaded data from the GDC Data Portal comes with one folder for each patient, and each folder has a .tsv file and a logs folder. 
# Therefore, we need to copy the .tsv files to one folder.

In [4]:
# Function to copy .tsv file to one folder
def copy_tsv_files(source_dir, destination_dir):
    for root, _, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.tsv'):
                source_file_path = os.path.join(root, file)
                destination_file_path = os.path.join(destination_dir, file)
                shutil.copy2(source_file_path, destination_file_path)

source_directory = './GDC_Data'
destination_directory = './Raw_gene_count'
copy_tsv_files(source_directory, destination_directory)

In [7]:
# Raw count data sample
df = pd.read_csv('./Raw_gene_count/0a9e33db-2527-4cc3-8669-a7c10fed7a7f.rna_seq.augmented_star_gene_counts.tsv', delimiter='\t', header=1)
df.head()

,gene_id,gene_name,gene_type,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded
0,N_unmapped,NaN,NaN,5893106,5893106,5893106,NaN,NaN,NaN
1,N_multimapping,NaN,NaN,5181136,5181136,5181136,NaN,NaN,NaN
2,N_noFeature,NaN,NaN,3472993,40032060,40254206,NaN,NaN,NaN
3,N_ambiguous,NaN,NaN,7106268,1730200,1728819,NaN,NaN,NaN
4,ENSG00000000003.15,TSPAN6,protein_coding,5729,2888,2841,72.3741,18.3759,18.0361


In [8]:
# Combine all gene count files into one file
folder_path = '/Users/tk/Desktop/Research-CITS5014/Raw_gene_count'

# Initialize an empty DataFrame
merged_df = pd.DataFrame()

# Loop over all files in the directory
for filename in os.listdir(folder_path):
    if filename.endswith('.tsv'):
        file_path = os.path.join(folder_path, filename)
        # Read the file
        df = pd.read_csv(file_path, delimiter='\t', header=1) 
        # Keep only the required columns
        df = df[['gene_id', 'unstranded']]
        # Rename columns
        df.columns = ['gene_id', filename] # using file name as patient ID first
        # If merged_df is empty, set it to df
        if merged_df.empty:
            merged_df = df
        else:
            # Merge the new DataFrame with the previous merged DataFrame
            merged_df = pd.merge(merged_df, df, on='gene_id', how='inner')

# Remove first 4 rows
merged_df = merged_df.iloc[4:]  
# Set the first column (gene ID) as the index
merged_df.set_index(merged_df.columns[0], inplace=True)
merged_df.index.name = None

In [10]:
merged_df

,7be7033e-db49-45cd-8ea5-827266707b1e.rna_seq.augmented_star_gene_counts.tsv,da549530-7ba5-4302-acd2-40a766860216.rna_seq.augmented_star_gene_counts.tsv,fc70931f-4a97-4538-b055-8ed988ab601c.rna_seq.augmented_star_gene_counts.tsv,cf3fe851-479f-4d22-9daf-99b163d0ed06.rna_seq.augmented_star_gene_counts.tsv,66941b24-9c8f-4657-a4eb-8cc267e38bdc.rna_seq.augmented_star_gene_counts.tsv,488d0d81-4872-4204-b8e8-745e64a35d65.rna_seq.augmented_star_gene_counts.tsv,0b59b278-d2f6-4aa6-bf6d-b8d743ae83af.rna_seq.augmented_star_gene_counts.tsv,fe379cd5-d51b-44b3-9adf-49e2cc3b0bf5.rna_seq.augmented_star_gene_counts.tsv,9eb69ff7-492b-4ed3-8762-c25c3dffa1fe.rna_seq.augmented_star_gene_counts.tsv,66116a4d-210d-45f8-8dba-3f8c79566ecc.rna_seq.augmented_star_gene_counts.tsv,...,3334577a-8045-4613-9a1b-e79fd62d5d6d.rna_seq.augmented_star_gene_counts.tsv,5c957f54-8c46-4f13-a22b-49c175e8af3a.rna_seq.augmented_star_gene_counts.tsv,2b5864c9-7577-4e93-8d22-1f45a98267f1.rna_seq.augmented_star_gene_counts.tsv,c6151c87-ab96-49c3-a250-b56211cd20e5.rna_seq.augmented_star_gene_counts.tsv,619785ac-0041-4b26-a887-d8ca021eaab1.rna_seq.augmented_star_gene_counts.tsv,9e785594-f00c-4805-8901-66c73fe2588d.rna_seq.augmented_star_gene_counts.tsv,ba6e6579-8858-467a-9d0b-b8df38cd5d62.rna_seq.augmented_star_gene_counts.tsv,466a4121-0583-47ee-9d54-1f1c2d47e381.rna_seq.augmented_star_gene_counts.tsv,4d4d40f2-1f96-4d31-81aa-d44b17a217ed.rna_seq.augmented_star_gene_counts.tsv,956570e5-7ed4-4d17-a830-f1bc67a7af15.rna_seq.augmented_star_gene_counts.tsv
ENSG00000000003.15,2423,3820,1021,3743,4901,3974,1637,3101,3734,1451,...,1969,1211,2836,213,7144,1982,835,6854,3298,2973
ENSG00000000005.6,875,105,87,10,2,25,11,27,2,4,...,0,0,33,6,53,2,9,318,11,4
ENSG00000000419.13,1244,1705,1725,2321,2691,2931,3489,1597,3222,1954,...,2889,1537,1886,3135,2330,2444,312,3435,4596,3545
ENSG00000000457.14,554,1678,2043,507,2350,2639,1515,1849,1373,1866,...,737,734,1473,1857,1996,761,224,2107,1898,1675
ENSG00000000460.17,163,292,573,387,791,2118,1766,403,2180,417,...,715,756,937,401,642,845,61,698,2984,1188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288669.1,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
ENSG00000288670.1,287,399,304,96,370,975,523,238,982,315,...,534,183,495,297,501,172,89,653,566,707
ENSG00000288671.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000288674.1,18,3,6,5,4,9,5,12,6,5,...,2,6,0,2,9,4,9,9,6,7


In [11]:
merged_df_backup = merged_df

In [12]:
# Read sample sheet to map sample id
sample_sheet = pd.read_csv('./GDC_Data/gdc_sample_sheet.2024-03-26.tsv', delimiter='\t')

# Filter rows
sample_sheet = sample_sheet[(sample_sheet['Data Type'] == 'Gene Expression Quantification') & (sample_sheet['Sample Type'] == 'Primary Tumor')]

# Apply the mapping to rename columns in merged_df
file_name_to_id = sample_sheet.set_index('File Name')['Sample ID'].to_dict()
merged_df.rename(columns=file_name_to_id, inplace=True)
merged_df

,7be7033e-db49-45cd-8ea5-827266707b1e.rna_seq.augmented_star_gene_counts.tsv,da549530-7ba5-4302-acd2-40a766860216.rna_seq.augmented_star_gene_counts.tsv,TCGA-BH-A0DX-01A,TCGA-E9-A1NC-01A,TCGA-A2-A0CT-01A,TCGA-BH-A0C7-01B,TCGA-AC-A6IW-01A,TCGA-AC-A3W5-01A,TCGA-AO-A0J6-01A,TCGA-BH-A0HI-01A,...,TCGA-A2-A0CM-01A,TCGA-OL-A5RW-01A,TCGA-BH-A0E0-01A,TCGA-D8-A1JS-01A,TCGA-A2-A0YI-01A,TCGA-A7-A4SD-01A,TCGA-AO-A0JB-01A,466a4121-0583-47ee-9d54-1f1c2d47e381.rna_seq.augmented_star_gene_counts.tsv,TCGA-AR-A1AY-01A,TCGA-AR-A0TZ-01A
ENSG00000000003.15,2423,3820,1021,3743,4901,3974,1637,3101,3734,1451,...,1969,1211,2836,213,7144,1982,835,6854,3298,2973
ENSG00000000005.6,875,105,87,10,2,25,11,27,2,4,...,0,0,33,6,53,2,9,318,11,4
ENSG00000000419.13,1244,1705,1725,2321,2691,2931,3489,1597,3222,1954,...,2889,1537,1886,3135,2330,2444,312,3435,4596,3545
ENSG00000000457.14,554,1678,2043,507,2350,2639,1515,1849,1373,1866,...,737,734,1473,1857,1996,761,224,2107,1898,1675
ENSG00000000460.17,163,292,573,387,791,2118,1766,403,2180,417,...,715,756,937,401,642,845,61,698,2984,1188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288669.1,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
ENSG00000288670.1,287,399,304,96,370,975,523,238,982,315,...,534,183,495,297,501,172,89,653,566,707
ENSG00000288671.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000288674.1,18,3,6,5,4,9,5,12,6,5,...,2,6,0,2,9,4,9,9,6,7


In [13]:
# Remove Normal (non-cancer) sample
raw_count = merged_df.loc[:, merged_df.columns.str.startswith('TCGA')]

# Drop duplicate columns, keeping the first occurrence
raw_count = raw_count.loc[:, ~raw_count.columns.duplicated()]
raw_count

,TCGA-BH-A0DX-01A,TCGA-E9-A1NC-01A,TCGA-A2-A0CT-01A,TCGA-BH-A0C7-01B,TCGA-AC-A6IW-01A,TCGA-AC-A3W5-01A,TCGA-AO-A0J6-01A,TCGA-BH-A0HI-01A,TCGA-A8-A07S-01A,TCGA-A1-A0SH-01A,...,TCGA-LL-A441-01A,TCGA-A2-A0CM-01A,TCGA-OL-A5RW-01A,TCGA-BH-A0E0-01A,TCGA-D8-A1JS-01A,TCGA-A2-A0YI-01A,TCGA-A7-A4SD-01A,TCGA-AO-A0JB-01A,TCGA-AR-A1AY-01A,TCGA-AR-A0TZ-01A
ENSG00000000003.15,1021,3743,4901,3974,1637,3101,3734,1451,1800,3691,...,5172,1969,1211,2836,213,7144,1982,835,3298,2973
ENSG00000000005.6,87,10,2,25,11,27,2,4,7,3,...,68,0,0,33,6,53,2,9,11,4
ENSG00000000419.13,1725,2321,2691,2931,3489,1597,3222,1954,1582,1518,...,4333,2889,1537,1886,3135,2330,2444,312,4596,3545
ENSG00000000457.14,2043,507,2350,2639,1515,1849,1373,1866,2861,941,...,2584,737,734,1473,1857,1996,761,224,1898,1675
ENSG00000000460.17,573,387,791,2118,1766,403,2180,417,779,320,...,971,715,756,937,401,642,845,61,2984,1188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288669.1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
ENSG00000288670.1,304,96,370,975,523,238,982,315,830,255,...,395,534,183,495,297,501,172,89,566,707
ENSG00000288671.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000288674.1,6,5,4,9,5,12,6,5,4,4,...,35,2,6,0,2,9,4,9,6,7


In [15]:
# Create a mapping dictionary for surrogate keys
surrogate_mapping = {value: idx for idx, value in enumerate(raw_count.columns)}
surrogate_mapping

{'TCGA-BH-A0DX-01A': 0,
 'TCGA-E9-A1NC-01A': 1,
 'TCGA-A2-A0CT-01A': 2,
 'TCGA-BH-A0C7-01B': 3,
 'TCGA-AC-A6IW-01A': 4,
 'TCGA-AC-A3W5-01A': 5,
 'TCGA-AO-A0J6-01A': 6,
 'TCGA-BH-A0HI-01A': 7,
 'TCGA-A8-A07S-01A': 8,
 'TCGA-A1-A0SH-01A': 9,
 'TCGA-GM-A2DN-01A': 10,
 'TCGA-BH-A204-01A': 11,
 'TCGA-A2-A4S3-01A': 12,
 'TCGA-E2-A15T-01A': 13,
 'TCGA-AN-A0XV-01A': 14,
 'TCGA-A7-A0CH-01A': 15,
 'TCGA-JL-A3YW-01A': 16,
 'TCGA-BH-A0HQ-01A': 17,
 'TCGA-BH-A0BC-01A': 18,
 'TCGA-PE-A5DC-01A': 19,
 'TCGA-BH-A1FG-01A': 20,
 'TCGA-OL-A66N-01A': 21,
 'TCGA-EW-A1IW-01A': 22,
 'TCGA-BH-A0H9-01A': 23,
 'TCGA-B6-A0RS-01A': 24,
 'TCGA-BH-A0EE-01A': 25,
 'TCGA-B6-A0IJ-01A': 26,
 'TCGA-AC-A2FF-01A': 27,
 'TCGA-A2-A0EP-01A': 28,
 'TCGA-BH-A0B8-01A': 29,
 'TCGA-3C-AALI-01A': 30,
 'TCGA-BH-A1FU-01A': 31,
 'TCGA-A7-A3J0-01A': 32,
 'TCGA-BH-A18I-01A': 33,
 'TCGA-BH-A0C0-01A': 34,
 'TCGA-EW-A1OV-01A': 35,
 'TCGA-GM-A3NY-01A': 36,
 'TCGA-EW-A1OZ-01A': 37,
 'TCGA-BH-A18F-01A': 38,
 'TCGA-A2-A25B-01A': 39,
 'TCGA-A8-

#### Create Metadata file containing case_id, survival_time, sample_id, status, sample_id_key, subtype

In [40]:
# Read survival time data
clinical = pd.read_csv('./GDC_Data/clinical.project-tcga-brca.2024-03-19/clinical.tsv', delimiter='\t')

# Each case_submitter_id has 2 records
clinical = clinical.drop_duplicates(subset='case_submitter_id')
clinical.head()

,case_id,case_submitter_id,project_id,age_at_index,age_is_obfuscated,cause_of_death,cause_of_death_source,country_of_residence_at_enrollment,days_to_birth,days_to_death,...,treatment_arm,treatment_dose,treatment_dose_units,treatment_effect,treatment_effect_indicator,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,001cef41-ff86-4d3f-a140-a647ac4b10a1,TCGA-E2-A1IU,TCGA-BRCA,60,'--,'--,'--,'--,-22279,'--,...,'--,'--,'--,'--,'--,'--,'--,no,'--,"Radiation Therapy, NOS"
2,0045349c-69d9-4306-a403-c9c1fa836644,TCGA-A1-A0SB,TCGA-BRCA,70,'--,'--,'--,'--,-25833,'--,...,'--,'--,'--,'--,'--,'--,'--,not reported,'--,"Radiation Therapy, NOS"
4,00807dae-9f4a-4fd1-aac2-82eb11bf2afb,TCGA-A2-A04W,TCGA-BRCA,50,'--,'--,'--,'--,-18345,'--,...,'--,'--,'--,'--,'--,'--,'--,no,'--,"Radiation Therapy, NOS"
6,00a2d166-78c9-4687-a195-3d6315c27574,TCGA-AN-A0AM,TCGA-BRCA,56,'--,'--,'--,'--,-20713,'--,...,'--,'--,'--,'--,'--,'--,'--,no,'--,"Pharmaceutical Therapy, NOS"
8,00b11ca8-8540-4a3d-b602-ec754b00230b,TCGA-LL-A440,TCGA-BRCA,61,'--,'--,'--,'--,-22497,'--,...,'--,'--,'--,'--,'--,'--,'--,yes,'--,"Pharmaceutical Therapy, NOS"


In [41]:
# Check duplicate
survival_time = clinical[['case_submitter_id', 'days_to_death']]
duplicated_caseid = survival_time[survival_time.duplicated(subset='case_submitter_id', keep=False)]
len(duplicated_caseid)

0

In [42]:
# Create meta data
meta_data = pd.merge(survival_time, sample_sheet[['Case ID', 'Sample ID']], left_on='case_submitter_id', right_on='Case ID', how='inner')
meta_data = meta_data.drop_duplicates(subset='Sample ID')

# Add status column
meta_data['status'] = meta_data.apply(lambda row: 1 if row['days_to_death'] == "'--" else 0, axis=1) # 1 for alive, 0 for death
meta_data

,case_submitter_id,days_to_death,Case ID,Sample ID,status
0,TCGA-E2-A1IU,'--,TCGA-E2-A1IU,TCGA-E2-A1IU-01A,1
1,TCGA-A1-A0SB,'--,TCGA-A1-A0SB,TCGA-A1-A0SB-01A,1
2,TCGA-A2-A04W,'--,TCGA-A2-A04W,TCGA-A2-A04W-01A,1
3,TCGA-AN-A0AM,'--,TCGA-AN-A0AM,TCGA-AN-A0AM-01A,1
4,TCGA-LL-A440,'--,TCGA-LL-A440,TCGA-LL-A440-01A,1
...,...,...,...,...,...
1106,TCGA-A2-A0CP,'--,TCGA-A2-A0CP,TCGA-A2-A0CP-01A,1
1107,TCGA-PL-A8LX,'--,TCGA-PL-A8LX,TCGA-PL-A8LX-01A,1
1108,TCGA-A2-A3XZ,'--,TCGA-A2-A3XZ,TCGA-A2-A3XZ-01A,1
1109,TCGA-E9-A295,'--,TCGA-E9-A295,TCGA-E9-A295-01A,1


In [43]:
# Find maximum days to death
days_to_death = meta_data['days_to_death']
grouped = days_to_death.groupby(days_to_death.eq("'--"))
group_alive = grouped.get_group(True)
group_death = grouped.get_group(False)
group_death = pd.to_numeric(group_death)
days_survive = group_death.max()

# Replace '-- with the maximum days survived
meta_data['days_to_death'] = meta_data['days_to_death'].replace("'--", days_survive)
meta_data['days_to_death'] = meta_data['days_to_death'].astype('int')

In [44]:
# Drop Case ID column
meta_data = meta_data.drop(meta_data.columns[2], axis=1)
meta_data.columns = ['case_id', 'survival_time', 'sample_id', 'status']

# Map the values in column 'sample_id' to the surrogate keys
meta_data['sample_id_key'] = meta_data['sample_id'].map(surrogate_mapping)
meta_data

,case_id,survival_time,sample_id,status,sample_id_key
0,TCGA-E2-A1IU,7455,TCGA-E2-A1IU-01A,1,764
1,TCGA-A1-A0SB,7455,TCGA-A1-A0SB-01A,1,1092
2,TCGA-A2-A04W,7455,TCGA-A2-A04W-01A,1,742
3,TCGA-AN-A0AM,7455,TCGA-AN-A0AM-01A,1,668
4,TCGA-LL-A440,7455,TCGA-LL-A440-01A,1,783
...,...,...,...,...,...
1106,TCGA-A2-A0CP,7455,TCGA-A2-A0CP-01A,1,942
1107,TCGA-PL-A8LX,7455,TCGA-PL-A8LX-01A,1,418
1108,TCGA-A2-A3XZ,7455,TCGA-A2-A3XZ-01A,1,251
1109,TCGA-E9-A295,7455,TCGA-E9-A295-01A,1,136


In [45]:
# Drop sample with survival time = 0
cols_to_drop = meta_data.loc[meta_data['survival_time'] == 0, 'sample_id']
raw_count = raw_count.drop(columns=[col for col in cols_to_drop if col in raw_count.columns])
meta_data = meta_data[~meta_data['sample_id'].isin(cols_to_drop.to_list())]

print(raw_count.shape)
print(meta_data.shape)
# There are 2 sample with survival time = 0

(60660, 1104)
(1104, 5)


In [46]:
# Read BRCA subtype data
brca_pam50 = pd.read_excel('./brca_pam50_ihc.xlsx', header=2)
brca_pam50 = brca_pam50[['Case.ID', 'PAM50']]
brca_pam50.columns = ['case_id', 'subtype']
meta_data = pd.merge(meta_data, brca_pam50, on='case_id', how='inner')
meta_data

/Users/tk/opt/anaconda3/envs/gene/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,case_id,survival_time,sample_id,status,sample_id_key,subtype
0,TCGA-E2-A1IU,7455,TCGA-E2-A1IU-01A,1,764,LumA
1,TCGA-A1-A0SB,7455,TCGA-A1-A0SB-01A,1,1092,Normal
2,TCGA-A2-A04W,7455,TCGA-A2-A04W-01A,1,742,Her2
3,TCGA-AN-A0AM,7455,TCGA-AN-A0AM-01A,1,668,LumB
4,TCGA-LL-A440,7455,TCGA-LL-A440-01A,1,783,LumA
...,...,...,...,...,...,...
820,TCGA-BH-A0HP,7455,TCGA-BH-A0HP-01A,1,821,LumA
821,TCGA-A2-A0CP,7455,TCGA-A2-A0CP-01A,1,942,LumA
822,TCGA-A2-A3XZ,7455,TCGA-A2-A3XZ-01A,1,251,Her2
823,TCGA-E9-A295,7455,TCGA-E9-A295-01A,1,136,LumA


In [48]:
duplicated_caseid = meta_data[meta_data.duplicated(subset='case_id', keep=False)]
duplicated_caseid
# These are duplicated case IDs. However, this won't be a problem because we will use sample IDs as unique identifiers.

,case_id,survival_time,sample_id,status,sample_id_key,subtype
5,TCGA-A7-A26E,7455,TCGA-A7-A26E-01A,1,250,LumA
6,TCGA-A7-A26E,7455,TCGA-A7-A26E-01B,1,248,LumA
202,TCGA-A7-A26F,7455,TCGA-A7-A26F-01A,1,464,Basal
203,TCGA-A7-A26F,7455,TCGA-A7-A26F-01B,1,630,Basal
336,TCGA-AC-A2QH,7455,TCGA-AC-A2QH-01B,1,52,Basal
337,TCGA-AC-A2QH,7455,TCGA-AC-A2QH-01A,1,95,Basal
354,TCGA-A7-A26J,7455,TCGA-A7-A26J-01A,1,789,LumA
355,TCGA-A7-A26J,7455,TCGA-A7-A26J-01B,1,997,LumA
356,TCGA-A7-A13G,7455,TCGA-A7-A13G-01A,1,315,LumA
357,TCGA-A7-A13G,7455,TCGA-A7-A13G-01B,1,848,LumA


In [49]:
# Check empty subtype column
meta_data['subtype'].isna().sum()

0

In [57]:
# Keep only sample with subtype information (we have to remove 1104-825 = 279 samples)
col_to_keep = meta_data.iloc[:,2].to_list()
print(len(col_to_keep))

825


In [58]:
raw_count = raw_count[col_to_keep]
raw_count

,TCGA-E2-A1IU-01A,TCGA-A1-A0SB-01A,TCGA-A2-A04W-01A,TCGA-AN-A0AM-01A,TCGA-LL-A440-01A,TCGA-A7-A26E-01A,TCGA-A7-A26E-01B,TCGA-A8-A07W-01A,TCGA-D8-A1XY-01A,TCGA-EW-A1P7-01A,...,TCGA-AC-A3EH-01A,TCGA-E2-A14O-01A,TCGA-E9-A249-01A,TCGA-BH-A0DS-01A,TCGA-D8-A147-01A,TCGA-BH-A0HP-01A,TCGA-A2-A0CP-01A,TCGA-A2-A3XZ-01A,TCGA-E9-A295-01A,TCGA-C8-A26W-01A
ENSG00000000003.15,850,4148,2718,2501,2842,5461,691,911,1983,1799,...,3101,2006,6296,2150,2449,6961,1234,2769,3352,4241
ENSG00000000005.6,5,604,1,1,113,12,20,22,5,95,...,1,14,8,32,2,56,14,9,18,13
ENSG00000000419.13,1680,1194,1914,7444,1617,2384,335,4349,2991,1602,...,1714,2664,3925,1169,3473,1951,863,1944,1538,3226
ENSG00000000457.14,1559,996,677,2268,1358,4063,1292,1420,2390,949,...,2492,1201,3212,1716,1632,2388,431,3211,1100,1812
ENSG00000000460.17,402,337,291,1177,408,1240,536,1197,582,308,...,931,564,1574,288,2145,872,113,484,380,1306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288669.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,1
ENSG00000288670.1,231,218,121,432,332,522,222,575,432,283,...,666,612,1151,382,667,482,164,306,193,441
ENSG00000288671.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000288674.1,3,11,1,11,28,16,3,4,5,6,...,14,1,31,13,14,9,4,16,1,5


In [59]:
# Rename the columns using the surrogate_mapping
raw_count.columns = [surrogate_mapping[col] for col in raw_count.columns]
raw_count

,764,1092,742,668,783,250,248,1023,156,614,...,626,445,428,535,887,821,942,251,136,173
ENSG00000000003.15,850,4148,2718,2501,2842,5461,691,911,1983,1799,...,3101,2006,6296,2150,2449,6961,1234,2769,3352,4241
ENSG00000000005.6,5,604,1,1,113,12,20,22,5,95,...,1,14,8,32,2,56,14,9,18,13
ENSG00000000419.13,1680,1194,1914,7444,1617,2384,335,4349,2991,1602,...,1714,2664,3925,1169,3473,1951,863,1944,1538,3226
ENSG00000000457.14,1559,996,677,2268,1358,4063,1292,1420,2390,949,...,2492,1201,3212,1716,1632,2388,431,3211,1100,1812
ENSG00000000460.17,402,337,291,1177,408,1240,536,1197,582,308,...,931,564,1574,288,2145,872,113,484,380,1306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288669.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,1
ENSG00000288670.1,231,218,121,432,332,522,222,575,432,283,...,666,612,1151,382,667,482,164,306,193,441
ENSG00000288671.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000288674.1,3,11,1,11,28,16,3,4,5,6,...,14,1,31,13,14,9,4,16,1,5


In [60]:
raw_count.to_csv('./csv/raw_count.csv', index=True)
meta_data.to_csv('./csv/meta_data.csv', index=False)

### Gene Normalisation

In [64]:
raw_count = pd.read_csv('./csv/raw_count.csv')
raw_count.set_index(raw_count.columns[0], inplace=True)
raw_count.index.name = None
raw_count = raw_count.astype(float)

In [65]:
# Count the number of genes where all counts are zero
zero_rows_count = (raw_count == 0).all(axis=1).sum()
print(zero_rows_count)

2702


In [66]:
# Gene count normalisation
housekeeping_genes = ["ENSG00000111640.15", "ENSG00000075624.17", "ENSG00000134644.16", "ENSG00000150991.15"]
norm_data = raw_count.copy()

# Data is normalised by the mean of the housekeeping genes per individual (column)
for col in norm_data.columns:
    sm: float = 0
    cnt = 0
    # Go through rows
    for gene in housekeeping_genes:
        if gene not in norm_data.index:
            print(gene, " not in index for ", col)
            continue
        sm += norm_data.at[gene, col]
        cnt += 1

    mean = sm / cnt
    # Normalize all rows
    norm_data.loc[:, col] = norm_data.loc[:, col] / mean
print(norm_data.shape)

(60660, 825)


In [67]:
norm_data

,764,1092,742,668,783,250,248,1023,156,614,...,626,445,428,535,887,821,942,251,136,173
ENSG00000000003.15,0.017497,0.049352,0.017216,0.018105,0.025710,0.041696,0.048345,0.007936,0.022253,0.011058,...,0.019830,0.014368,0.051851,0.019578,0.027949,0.063514,0.013882,0.021598,0.052519,0.028860
ENSG00000000005.6,0.000103,0.007186,0.000006,0.000007,0.001022,0.000092,0.001399,0.000192,0.000056,0.000584,...,0.000006,0.000100,0.000066,0.000291,0.000023,0.000511,0.000157,0.000070,0.000282,0.000088
ENSG00000000419.13,0.034583,0.014206,0.012124,0.053887,0.014628,0.018202,0.023438,0.037888,0.033565,0.009847,...,0.010961,0.019081,0.032324,0.010645,0.039635,0.017801,0.009708,0.015163,0.024097,0.021953
ENSG00000000457.14,0.032092,0.011850,0.004288,0.016418,0.012285,0.031022,0.090394,0.012371,0.026821,0.005833,...,0.015936,0.008602,0.026452,0.015626,0.018625,0.021789,0.004848,0.025046,0.017235,0.012331
ENSG00000000460.17,0.008275,0.004010,0.001843,0.008520,0.003691,0.009468,0.037501,0.010428,0.006531,0.001893,...,0.005954,0.004040,0.012963,0.002623,0.024480,0.007956,0.001271,0.003775,0.005954,0.008887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288669.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000022,0.000000,0.000000,0.000007
ENSG00000288670.1,0.004755,0.002594,0.000766,0.003127,0.003003,0.003986,0.015532,0.005009,0.004848,0.001740,...,0.004259,0.004384,0.009479,0.003479,0.007612,0.004398,0.001845,0.002387,0.003024,0.003001
ENSG00000288671.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000288674.1,0.000062,0.000131,0.000006,0.000080,0.000253,0.000122,0.000210,0.000035,0.000056,0.000037,...,0.000090,0.000007,0.000255,0.000118,0.000160,0.000082,0.000045,0.000125,0.000016,0.000034


In [68]:
# Apply ln(x+1) transformation to the data. This is a normalisation step
norm_data = np.log(norm_data + 1)
norm_data = norm_data.round(6)
norm_data

,764,1092,742,668,783,250,248,1023,156,614,...,626,445,428,535,887,821,942,251,136,173
ENSG00000000003.15,0.017346,0.048173,0.017070,0.017943,0.025385,0.040850,0.047213,0.007905,0.022009,0.010997,...,0.019636,0.014266,0.050551,0.019389,0.027566,0.061578,0.013786,0.021368,0.051187,0.028451
ENSG00000000005.6,0.000103,0.007161,0.000006,0.000007,0.001022,0.000092,0.001398,0.000192,0.000056,0.000584,...,0.000006,0.000100,0.000066,0.000291,0.000023,0.000511,0.000157,0.000070,0.000282,0.000088
ENSG00000000419.13,0.033998,0.014106,0.012051,0.052486,0.014522,0.018039,0.023168,0.037188,0.033014,0.009799,...,0.010901,0.018902,0.031813,0.010589,0.038870,0.017645,0.009661,0.015049,0.023812,0.021715
ENSG00000000457.14,0.031588,0.011781,0.004279,0.016285,0.012210,0.030551,0.086539,0.012295,0.026467,0.005816,...,0.015810,0.008566,0.026109,0.015505,0.018454,0.021555,0.004837,0.024737,0.017088,0.012255
ENSG00000000460.17,0.008241,0.004002,0.001842,0.008484,0.003684,0.009423,0.036815,0.010374,0.006510,0.001891,...,0.005936,0.004032,0.012879,0.002619,0.024185,0.007925,0.001270,0.003768,0.005936,0.008848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288669.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000022,0.000000,0.000000,0.000007
ENSG00000288670.1,0.004744,0.002590,0.000766,0.003122,0.002999,0.003978,0.015413,0.004997,0.004836,0.001738,...,0.004250,0.004374,0.009434,0.003473,0.007583,0.004388,0.001843,0.002384,0.003019,0.002996
ENSG00000288671.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000288674.1,0.000062,0.000131,0.000006,0.000080,0.000253,0.000122,0.000210,0.000035,0.000056,0.000037,...,0.000090,0.000007,0.000255,0.000118,0.000160,0.000082,0.000045,0.000125,0.000016,0.000034


In [70]:
norm_data.to_csv('./csv/norm_data.csv', index=True)

### Create unified data with all the information

In [4]:
# Read .csv files
norm_data = pd.read_csv('./csv/norm_data.csv')
meta_data = pd.read_csv('./csv/meta_data.csv')

norm_data.set_index(norm_data.columns[0], inplace=True)
norm_data.index.name = None
norm_data.columns = norm_data.columns.astype(int)

# Creat mapping dictionary
key_to_time = meta_data.set_index('sample_id_key')['survival_time'].to_dict()
key_to_time = {int(key): int(value) for key, value in key_to_time.items()}
key_to_status = meta_data.set_index('sample_id_key')['status'].to_dict()
key_to_subtype = meta_data.set_index('sample_id_key')['subtype'].to_dict()
key_to_id = meta_data.set_index('sample_id_key')['sample_id'].to_dict()

In [5]:
# Convert the dictionary to a DataFrame
survival_time = pd.DataFrame([key_to_time], index=['days_to_death'])
status = pd.DataFrame([key_to_status], index=['status'])
subtype = pd.DataFrame([key_to_subtype], index=['PAM50'])

In [6]:
# Append the new DataFrame to norm_data
df_combined = pd.concat([norm_data, survival_time])
df_combined = pd.concat([df_combined, status])
df_combined = pd.concat([df_combined, subtype])
df_combined

,764,1092,742,668,783,250,248,1023,156,614,...,626,445,428,535,887,821,942,251,136,173
ENSG00000000003.15,0.017346,0.048173,0.01707,0.017943,0.025385,0.04085,0.047213,0.007905,0.022009,0.010997,...,0.019636,0.014266,0.050551,0.019389,0.027566,0.061578,0.013786,0.021368,0.051187,0.028451
ENSG00000000005.6,0.000103,0.007161,0.000006,0.000007,0.001022,0.000092,0.001398,0.000192,0.000056,0.000584,...,0.000006,0.0001,0.000066,0.000291,0.000023,0.000511,0.000157,0.00007,0.000282,0.000088
ENSG00000000419.13,0.033998,0.014106,0.012051,0.052486,0.014522,0.018039,0.023168,0.037188,0.033014,0.009799,...,0.010901,0.018902,0.031813,0.010589,0.03887,0.017645,0.009661,0.015049,0.023812,0.021715
ENSG00000000457.14,0.031588,0.011781,0.004279,0.016285,0.01221,0.030551,0.086539,0.012295,0.026467,0.005816,...,0.01581,0.008566,0.026109,0.015505,0.018454,0.021555,0.004837,0.024737,0.017088,0.012255
ENSG00000000460.17,0.008241,0.004002,0.001842,0.008484,0.003684,0.009423,0.036815,0.010374,0.00651,0.001891,...,0.005936,0.004032,0.012879,0.002619,0.024185,0.007925,0.00127,0.003768,0.005936,0.008848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288674.1,0.000062,0.000131,0.000006,0.00008,0.000253,0.000122,0.00021,0.000035,0.000056,0.000037,...,0.00009,0.000007,0.000255,0.000118,0.00016,0.000082,0.000045,0.000125,0.000016,0.000034
ENSG00000288675.1,0.000124,0.000309,0.000215,0.000152,0.000416,0.000115,0.00049,0.000131,0.000224,0.000141,...,0.00023,0.000057,0.000214,0.000155,0.000388,0.000255,0.000416,0.000733,0.000157,0.000306
days_to_death,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,...,197.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0
status,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [7]:
# Rename columns to real sample_id
df_combined.rename(columns=key_to_id, inplace=True)
df_combined

,TCGA-E2-A1IU-01A,TCGA-A1-A0SB-01A,TCGA-A2-A04W-01A,TCGA-AN-A0AM-01A,TCGA-LL-A440-01A,TCGA-A7-A26E-01A,TCGA-A7-A26E-01B,TCGA-A8-A07W-01A,TCGA-D8-A1XY-01A,TCGA-EW-A1P7-01A,...,TCGA-AC-A3EH-01A,TCGA-E2-A14O-01A,TCGA-E9-A249-01A,TCGA-BH-A0DS-01A,TCGA-D8-A147-01A,TCGA-BH-A0HP-01A,TCGA-A2-A0CP-01A,TCGA-A2-A3XZ-01A,TCGA-E9-A295-01A,TCGA-C8-A26W-01A
ENSG00000000003.15,0.017346,0.048173,0.01707,0.017943,0.025385,0.04085,0.047213,0.007905,0.022009,0.010997,...,0.019636,0.014266,0.050551,0.019389,0.027566,0.061578,0.013786,0.021368,0.051187,0.028451
ENSG00000000005.6,0.000103,0.007161,0.000006,0.000007,0.001022,0.000092,0.001398,0.000192,0.000056,0.000584,...,0.000006,0.0001,0.000066,0.000291,0.000023,0.000511,0.000157,0.00007,0.000282,0.000088
ENSG00000000419.13,0.033998,0.014106,0.012051,0.052486,0.014522,0.018039,0.023168,0.037188,0.033014,0.009799,...,0.010901,0.018902,0.031813,0.010589,0.03887,0.017645,0.009661,0.015049,0.023812,0.021715
ENSG00000000457.14,0.031588,0.011781,0.004279,0.016285,0.01221,0.030551,0.086539,0.012295,0.026467,0.005816,...,0.01581,0.008566,0.026109,0.015505,0.018454,0.021555,0.004837,0.024737,0.017088,0.012255
ENSG00000000460.17,0.008241,0.004002,0.001842,0.008484,0.003684,0.009423,0.036815,0.010374,0.00651,0.001891,...,0.005936,0.004032,0.012879,0.002619,0.024185,0.007925,0.00127,0.003768,0.005936,0.008848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288674.1,0.000062,0.000131,0.000006,0.00008,0.000253,0.000122,0.00021,0.000035,0.000056,0.000037,...,0.00009,0.000007,0.000255,0.000118,0.00016,0.000082,0.000045,0.000125,0.000016,0.000034
ENSG00000288675.1,0.000124,0.000309,0.000215,0.000152,0.000416,0.000115,0.00049,0.000131,0.000224,0.000141,...,0.00023,0.000057,0.000214,0.000155,0.000388,0.000255,0.000416,0.000733,0.000157,0.000306
days_to_death,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,...,197.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0,7455.0
status,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [8]:
# Convert days_to_death, status to integers
df_combined.iloc[-3:-1] = df_combined.iloc[-3:-1].astype(int)
df_combined

,TCGA-E2-A1IU-01A,TCGA-A1-A0SB-01A,TCGA-A2-A04W-01A,TCGA-AN-A0AM-01A,TCGA-LL-A440-01A,TCGA-A7-A26E-01A,TCGA-A7-A26E-01B,TCGA-A8-A07W-01A,TCGA-D8-A1XY-01A,TCGA-EW-A1P7-01A,...,TCGA-AC-A3EH-01A,TCGA-E2-A14O-01A,TCGA-E9-A249-01A,TCGA-BH-A0DS-01A,TCGA-D8-A147-01A,TCGA-BH-A0HP-01A,TCGA-A2-A0CP-01A,TCGA-A2-A3XZ-01A,TCGA-E9-A295-01A,TCGA-C8-A26W-01A
ENSG00000000003.15,0.017346,0.048173,0.01707,0.017943,0.025385,0.04085,0.047213,0.007905,0.022009,0.010997,...,0.019636,0.014266,0.050551,0.019389,0.027566,0.061578,0.013786,0.021368,0.051187,0.028451
ENSG00000000005.6,0.000103,0.007161,0.000006,0.000007,0.001022,0.000092,0.001398,0.000192,0.000056,0.000584,...,0.000006,0.0001,0.000066,0.000291,0.000023,0.000511,0.000157,0.00007,0.000282,0.000088
ENSG00000000419.13,0.033998,0.014106,0.012051,0.052486,0.014522,0.018039,0.023168,0.037188,0.033014,0.009799,...,0.010901,0.018902,0.031813,0.010589,0.03887,0.017645,0.009661,0.015049,0.023812,0.021715
ENSG00000000457.14,0.031588,0.011781,0.004279,0.016285,0.01221,0.030551,0.086539,0.012295,0.026467,0.005816,...,0.01581,0.008566,0.026109,0.015505,0.018454,0.021555,0.004837,0.024737,0.017088,0.012255
ENSG00000000460.17,0.008241,0.004002,0.001842,0.008484,0.003684,0.009423,0.036815,0.010374,0.00651,0.001891,...,0.005936,0.004032,0.012879,0.002619,0.024185,0.007925,0.00127,0.003768,0.005936,0.008848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288674.1,0.000062,0.000131,0.000006,0.00008,0.000253,0.000122,0.00021,0.000035,0.000056,0.000037,...,0.00009,0.000007,0.000255,0.000118,0.00016,0.000082,0.000045,0.000125,0.000016,0.000034
ENSG00000288675.1,0.000124,0.000309,0.000215,0.000152,0.000416,0.000115,0.00049,0.000131,0.000224,0.000141,...,0.00023,0.000057,0.000214,0.000155,0.000388,0.000255,0.000416,0.000733,0.000157,0.000306
days_to_death,7455,7455,7455,7455,7455,7455,7455,7455,7455,7455,...,197,7455,7455,7455,7455,7455,7455,7455,7455,7455
status,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1


In [9]:
df_combined.iloc[-1] = df_combined.iloc[-1].replace('Normal', 'Normal-like')

In [10]:
df_combined.to_csv('./csv/unified_data.csv', index=True)